In [34]:
import pandas as pd
import numpy as np
import pulp

In [35]:
from functions import get_current_weekday, calculate_nfl_week, get_next_sunday, get_current_year

In [36]:
day = get_current_weekday()

In [37]:
date_string = get_next_sunday(day)

In [38]:
week = calculate_nfl_week(date_string)

In [39]:
season = get_current_year()

In [40]:
site_for_pred = input('Do you want to generate lineups for FanDuel or DraftKings? ')
if site_for_pred == 'FD':
    df = pd.read_csv('FD_predictions_' + str(season) + '_' + str(week) + '.csv')
else: 
    df = pd.read_csv('DK_predictions_' + str(season) + '_' + str(week) + '.csv')

Do you want to generate lineups for FanDuel or DraftKings?  DK


In [41]:
# dk_preds['Value'] = np.round((dk_preds['median_pred'] / dk_preds['salary']) * 1000, 3)

In [42]:
# dk_preds.sort_values(by = ['Value'], ascending = False)

In [43]:
# prob = pulp.LpProblem('DK_NFL_weekly', pulp.LpMaximize)
# player_vars = [pulp.LpVariable(f'player_{row.name}', cat='Binary') for row in df.itertuples()]

In [44]:
# total assigned players constraint
# prob += pulp.lpSum(player_var for player_var in player_vars) == 9
# # total salary constraint
# prob += pulp.lpSum(df.salary.iloc[i] * player_vars[i] for i in range(len(df))) <= 50000
# # create a helper function to return the number of players assigned each position
# def get_position_sum(player_vars, df, position):
#     return pulp.lpSum([player_vars[i] * (position in df['position'].iloc[i]) for i in range(len(df))])
    
# # for QB and DST, require 1 of each in the lineup
# prob += get_position_sum(player_vars, df, 'QB') == 1
# prob += get_position_sum(player_vars, df, 'D') == 1
# prob += get_position_sum(player_vars, df, 'TE') == 1
# # to account for the FLEX position, we allow additional selections of the 3 FLEX-eligible positions: RB, WR, TE
# prob += get_position_sum(player_vars, df, 'RB') >= 2
# prob += get_position_sum(player_vars, df, 'WR') >= 3


In [45]:
# prob += pulp.lpSum([df.max_pred.iloc[i] * player_vars[i] for i in range(len(df))])

In [46]:
# solve and print the status
# prob.solve()
# print(pulp.LpStatus[prob.status])

In [47]:
# total_salary_used = 0
# mean_pred_points = 0
# for i in range(len(df)):
#     if player_vars[i].value() == 1:
#         row = df.iloc[i]
#         print(row['position'], row['name'], row.team, row.salary, row.median_pred)
#         total_salary_used += row.salary
#         mean_pred_points += row.median_pred
# mean_pred_points /= 9  # divide by total players in roster to get a mean

# print(f"Total salary used: {total_salary_used}")
# print(f"Total predicted points: {mean_pred_points * 9}")  # Multiply mean by 9 to get total points


In [48]:
exclude_list = []

In [49]:
exclude = input('Do you want to exclude any players from lineups? ')
while exclude == 'y':
    player = input('Which player? ')
    exclude_list.append(player)
    exclude = input('Any other players? ')

if site_for_pred == 'FD':
    cap = 60000
else:
    cap = 50000

# Define the lineup generation function with a salary cap parameter
def generate_lineup(df, salary_cap, excluded_players=exclude_list):
    if excluded_players is None:
        excluded_players = []

    # Create the pulp problem
    prob = pulp.LpProblem('NFL_weekly', pulp.LpMaximize)

    # Create variables for each player indicating whether they are included in the lineup
    player_vars = [pulp.LpVariable(f'player_{row.name}', cat='Binary') for row in df.itertuples()]

    # Total assigned players constraint
    prob += pulp.lpSum(player_var for player_var in player_vars) == 9

    # Total salary constraint using the provided salary cap
    prob += pulp.lpSum(df.salary.iloc[i] * player_vars[i] for i in range(len(df))) <= salary_cap

    # Create a helper function to return the number of players assigned each position
    def get_position_sum(player_vars, df, position):
        return pulp.lpSum([player_vars[i] * (position in df['position'].iloc[i]) for i in range(len(df))])

    # Position constraints: 1 QB, 1 Defense, 1 TE, 2 RBs, 3 WRs, 1 FLEX (RB, WR, or TE)
    prob += get_position_sum(player_vars, df, 'QB') == 1
    prob += get_position_sum(player_vars, df, 'D') == 1
    prob += get_position_sum(player_vars, df, 'TE') == 1
    prob += get_position_sum(player_vars, df, 'RB') >= 2
    prob += get_position_sum(player_vars, df, 'WR') >= 3

    # Objective: Maximize total predicted points
    prob += pulp.lpSum([df.max_pred.iloc[i] * player_vars[i] for i in range(len(df))])

    # Exclude specific players
    for i, row in df.iterrows():
        if row['name'] in excluded_players:
            prob += player_vars[i] == 0  # Prevent this player from being selected

    # Solve the problem and print the status
    prob.solve()
    print(pulp.LpStatus[prob.status])

    total_salary_used = 0
    mean_pred_points = 0
    lineup = []

    # Gather lineup details
    for i in range(len(df)):
        if player_vars[i].value() == 1:
            row = df.iloc[i]
            lineup.append((row['position'], row['name'], row.team, row.salary, row.median_pred))
            total_salary_used += row.salary
            mean_pred_points += row.median_pred

    mean_pred_points /= 9  # Divide by total players in roster to get a mean

    # Print the lineup details
    for player in lineup:
        print(player)

    print(f"Total salary used: {total_salary_used}")
    print(f"Total predicted points: {mean_pred_points * 9}")  # Multiply mean by 9 to get total points

    return lineup

# Example of how to call the function for DraftKings

generate_lineup(df, cap, exclude_list)

generate = input('Do you want to generate any more lineups? ')
while generate == 'y':
    site_for_pred = input('Which site? ')
    if site_for_pred == 'FD':
        df = pd.read_csv('FD_predictions_' + str(season) + '_' + str(week) + '.csv')
        cap = 60000
    else: 
        df = pd.read_csv('DK_predictions_' + str(season) + '_' + str(week) + '.csv')
        cap = 50000
    exclude = input('Do you want to exclude any players from lineups? ')
    while exclude == 'y':
        player = input('Which player? ')
        exclude_list.append(player)
        exclude = input('Any other players? ')
    generate_lineup(df, cap, exclude_list)
    generate = input('Do you want to generate any more lineups? ')

Do you want to exclude any players from lineups?  y
Which player?  Tank Dell
Any other players?  n


Optimal
('QB', 'Kyler Murray', 'ARI', 6900, 21.87380790710449)
('RB', 'Alvin Kamara', 'NO', 7500, 19.410806604037347)
('WR', 'Chris Godwin', 'TB', 6500, 17.29442596435547)
('RB', 'James Conner', 'ARI', 6600, 16.56440541264341)
('WR', 'Malik Nabers', 'NYG', 6400, 16.050303919766336)
('TE', 'Brock Bowers', 'LV', 5400, 15.089193344116213)
('RB', 'Jaret Patterson', 'LAC', 4000, 11.685183450597531)
('WR', 'Devaughn Vele', 'DEN', 3400, 10.61919116973877)
('D', 'PIT', 'PIT', 3300, 10.12870979309082)
Total salary used: 50000
Total predicted points: 138.7160275654504


Do you want to generate any more lineups?  y
Which site?  DK
Do you want to exclude any players from lineups?  n


Optimal
('QB', 'Kyler Murray', 'ARI', 6900, 21.87380790710449)
('RB', 'Alvin Kamara', 'NO', 7500, 19.410806604037347)
('WR', 'Chris Godwin', 'TB', 6500, 17.29442596435547)
('RB', 'James Conner', 'ARI', 6600, 16.56440541264341)
('WR', 'Malik Nabers', 'NYG', 6400, 16.050303919766336)
('TE', 'Brock Bowers', 'LV', 5400, 15.089193344116213)
('RB', 'Jaret Patterson', 'LAC', 4000, 11.685183450597531)
('WR', 'Devaughn Vele', 'DEN', 3400, 10.61919116973877)
('D', 'PIT', 'PIT', 3300, 10.12870979309082)
Total salary used: 50000
Total predicted points: 138.7160275654504


Do you want to generate any more lineups?  n


In [ ]:
# generate_lineup(df, 50000, exclude)